In [35]:
import tensorflow as tf
import struct
import numpy as np

path = "../IAI_5/data/"

def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        if magic != 2051:
            raise ValueError(f"Invalid magic number {magic} in file: {file_path}")
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        images = image_data.reshape((num_images, rows, cols, 1))
        return images / 255.0

def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        if magic != 2049:
            raise ValueError(f"Invalid magic number {magic} in file: {file_path}")
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

# Load the MNIST dataset
train_images = load_mnist_images(path + 'train-images.idx3-ubyte')
train_labels = load_mnist_labels(path + 'train-labels.idx1-ubyte')
test_images = load_mnist_images(path + 't10k-images.idx3-ubyte')
test_labels = load_mnist_labels(path + 't10k-labels.idx1-ubyte')

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training the model
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.1)

# Evaluating the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")


Epoch 1/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8640 - loss: 0.4490 - val_accuracy: 0.9830 - val_loss: 0.0582
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9809 - loss: 0.0613 - val_accuracy: 0.9857 - val_loss: 0.0515
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9856 - loss: 0.0431 - val_accuracy: 0.9880 - val_loss: 0.0411
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9903 - loss: 0.0317 - val_accuracy: 0.9912 - val_loss: 0.0355
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9922 - loss: 0.0227 - val_accuracy: 0.9897 - val_loss: 0.0348
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9935 - loss: 0.0195 - val_accuracy: 0.9907 - val_loss: 0.0312
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9956 - loss: 0.0132 - val_accuracy: 0.9922 - val_loss: 0.0307
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9963 - loss: 0.0108 - val_accuracy: 0.